In [2]:
!pip install tensorflow
!pip install pillow
!pip install numpy
!pip install opencv-python

You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
You should consider upgrading via the 'pip install --upgrade pip' command.
     |████████████████████████████████| 28.7MB 8.0kB/s eta 0:00:01                               | 870kB 1.8MB/s eta 0:00:16     |███████▋                        | 6.8MB 1.8MB/s eta 0:00:13     |██████████▊                     | 9.6MB 3.3MB/s eta 0:00:06     |███████████▉                    | 10.6MB 3.3MB/s eta 0:00:06██████████████▎                 | 12.8MB 3.3MB/s eta 0:00:05     |█████████████████████████████   | 26.1MB 498kB/s eta 0:00:06
You should consider upgrading via the 'pip install --upgrade pip' command.


In [5]:
!pip install tensorflow --upgrade pip

     |████████████████████████████████| 109.2MB 2.7kB/s eta 0:00:01    |█████                           | 16.8MB 1.4MB/s eta 0:01:08     |█████▌                          | 18.8MB 1.5MB/s eta 0:01:02███▋                        | 26.1MB 568kB/s eta 0:02:27     |███████▉                        | 26.7MB 568kB/s eta 0:02:26███▌                  | 46.1MB 3.3MB/s eta 0:00:20     |██████████████▎                 | 48.8MB 1.5MB/s eta 0:00:40     |████████████████                | 54.5MB 611kB/s eta 0:01:30     |█████████████████               | 57.9MB 246kB/s eta 0:03:29     |███████████████████▏            | 65.3MB 177kB/s eta 0:04:08     |███████████████████▍            | 66.2MB 177kB/s eta 0:04:03     |████████████████████▌           | 70.1MB 982kB/s eta 0:00:40     |█████████████████████▋          | 73.6MB 153kB/s eta 0:03:52     |█████████████████████▊          | 74.2MB 153kB/s eta 0:03:48     |██████████████████████▋         | 77.0MB 229kB/s eta 0:02:21     |███████████████████████▎      

In [7]:
!pip install --upgrade pip

  Using cached https://files.pythonhosted.org/packages/30/db/9e38760b32e3e7f40cce46dd5fb107b8c73840df38f0046d8e6514e675a1/pip-19.2.3-py2.py3-none-any.whl
  Found existing installation: pip 19.2.2
    Uninstalling pip-19.2.2:
      Successfully uninstalled pip-19.2.2


In [9]:
!pip install tensorflow==2.0.0-rc0 

     |████████████████████████████████| 86.3MB 215kB/s eta 0:00:013   |▌                               | 1.3MB 2.7MB/s eta 0:00:32     |█████▋                          | 15.3MB 6.0MB/s eta 0:00:12     |████████▎                       | 22.3MB 5.0MB/s eta 0:00:13     |███████████▌                    | 31.0MB 9.6MB/s eta 0:00:06     |█████████████▋                  | 36.7MB 7.0MB/s eta 0:00:08     |█████████████▊                  | 37.0MB 7.0MB/s eta 0:00:08     |█████████████████████████▋      | 69.2MB 6.7MB/s eta 0:00:03ta 0:00:01
     |████████████████████████████████| 4.3MB 9.2MB/s eta 0:00:01     | 686kB 9.2MB/s eta 0:00:01
  Using cached https://files.pythonhosted.org/packages/71/e3/19762fdfc62877ae9102edf6342d71b28fbfd9dea3d2f96a882ce099b03f/Keras_Applications-1.0.8-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/d0/33/376510eb8d6246f3c30545f416b2263eee461e40940c2a4413c711bdf62d/google_pasta-0.1.7-py3-none-any.whl
     |████████████████████████████████| 71k

In [40]:
import tensorflow as tf
import os

graph_def = tf.GraphDef()
labels = []

# These are set to the default names from exported models, update as needed.
filename = "model.pb"
labels_filename = "labels.txt"

# Import the TF graph
with tf.gfile.GFile(filename, 'rb') as f:
    graph_def.ParseFromString(f.read())
    #tf.import_graph_def(graph_def, name='')

# Create a list of labels.
with open(labels_filename, 'rt') as lf:
    for l in lf:
        labels.append(l.strip())

In [41]:
def convert_to_opencv(image):
    # RGB -> BGR conversion is performed as well.
    image = image.convert('RGB')
    r,g,b = np.array(image).T
    opencv_image = np.array([b,g,r]).transpose()
    return opencv_image

def crop_center(img,cropx,cropy):
    h, w = img.shape[:2]
    startx = w//2-(cropx//2)
    starty = h//2-(cropy//2)
    return img[starty:starty+cropy, startx:startx+cropx]

def resize_down_to_1600_max_dim(image):
    h, w = image.shape[:2]
    if (h < 1600 and w < 1600):
        return image

    new_size = (1600 * w // h, 1600) if (h > w) else (1600, 1600 * h // w)
    return cv2.resize(image, new_size, interpolation = cv2.INTER_LINEAR)

def resize_to_256_square(image):
    h, w = image.shape[:2]
    return cv2.resize(image, (256, 256), interpolation = cv2.INTER_LINEAR)

def update_orientation(image):
    exif_orientation_tag = 0x0112
    if hasattr(image, '_getexif'):
        exif = image._getexif()
        if (exif != None and exif_orientation_tag in exif):
            orientation = exif.get(exif_orientation_tag, 1)
            # orientation is 1 based, shift to zero based and flip/transpose based on 0-based values
            orientation -= 1
            if orientation >= 4:
                image = image.transpose(Image.TRANSPOSE)
            if orientation == 2 or orientation == 3 or orientation == 6 or orientation == 7:
                image = image.transpose(Image.FLIP_TOP_BOTTOM)
            if orientation == 1 or orientation == 2 or orientation == 5 or orientation == 6:
                image = image.transpose(Image.FLIP_LEFT_RIGHT)
    return image

In [42]:
from PIL import Image
import numpy as np
import cv2

# Load from a file
imageFile = "int_download.jpg"
image = Image.open(imageFile)

# Update orientation based on EXIF tags, if the file has orientation info.
image = update_orientation(image)

# Convert to OpenCV format
image = convert_to_opencv(image)

In [43]:
# If the image has either w or h greater than 1600 we resize it down respecting
# aspect ratio such that the largest dimension is 1600
image = resize_down_to_1600_max_dim(image)

In [44]:
# These names are part of the model and cannot be changed.
output_layer = 'loss:0'
input_node = 'Placeholder:0'
predictions =''

with tf.Session() as sess:
    try:
        #print(tf.global_variables())
        prob_tensor = sess.graph.get_tensor_by_name(output_layer)
        predictions = sess.run(prob_tensor, {input_node: [augmented_image] })
        writer = tf.summary.FileWriter("output", sess.graph)
        print(sess.run(predictions))
        writer.close()
    except KeyError:
        print ("Couldn't find classification output layer: " + output_layer + ".")
        print ("Verify this a model exported from an Object Detection project.")
        exit(-1)
       # %tensorboard --logdir logs
#print(sess.graph.show())    

Couldn't find classification output layer: loss:0.
Verify this a model exported from an Object Detection project.


In [38]:
 #%tensorboard

In [33]:
#%tensorboard --logdir

UsageError: Line magic function `%tensorboard` not found.
